Estrae la cartella di input nella cartella local di Colab

In [ ]:
!unzip "/content/inputs.zip"

Archive:  /content/inputs.zip
   creating: inputs/
   creating: inputs/clean/
   creating: inputs/clean/chair/
  inflating: inputs/clean/chair/sedia1.jpg  
  inflating: inputs/clean/chair/sedia3.jpg  
   creating: inputs/clean/cup/
  inflating: inputs/clean/cup/tazza1.jpg  
  inflating: inputs/clean/cup/tazzaa2.jpg  
   creating: inputs/clean/lamp/
  inflating: inputs/clean/lamp/lampada1.jpg  
  inflating: inputs/clean/lamp/lampada2.jpg  
   creating: inputs/clean/sofa/
  inflating: inputs/clean/sofa/divano1.jpg  
  inflating: inputs/clean/sofa/divano3.jpg  
   creating: inputs/clean/table/
  inflating: inputs/clean/table/tavolo1.jpg  
  inflating: inputs/clean/table/tavolo2.jpg  
   creating: inputs/clean/vase/
  inflating: inputs/clean/vase/vaso2.jpg  
  inflating: inputs/clean/vase/vaso3.jpg  
   creating: inputs/real/
   creating: inputs/real/chair/
  inflating: inputs/real/chair/sedia1.jpg  
  inflating: inputs/real/chair/sedia2.jpg  
   creating: inputs/real/cup/
  inflating: inp

Installazione dei principali framework e package

In [ ]:
!pip install open-clip-torch
!pip install jaxtyping einops ftfy regex timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.9 MB/s eta 0:00:00


In [ ]:
import os,json

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:256"

from pathlib import Path
import numpy as np

import pandas as pd
from PIL import Image
import torchvision.transforms.v2.functional as tvf
import gc
import torch
import torch.nn.functional as F
from sphinx.ext.viewcode import OUTPUT_DIRNAME
from tqdm import tqdm
import random
#Todo togliere il commmento
from einops import rearrange


In [ ]:
torch.use_deterministic_algorithms(False)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

Impostazione del dispositivo di calcolo

In [ ]:
device = torch.device("cuda")

Clip_model per ottenere gli score delle generazioni

Definizioni dei Path


In [ ]:
from pathlib import Path

BASE_DATA_DIR=Path("/content")
INPUTS_PATH=BASE_DATA_DIR/"inputs"
OUT_DIR=Path("/content/outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)
SCORES_FILE_PATH=OUT_DIR/"clip_score.txt"

print('BASE_DATA_DIR    =', BASE_DATA_DIR)
print('INPUTS_PATH      =', INPUTS_PATH)
print('OUT_DIR          =', OUT_DIR)
print('SCORES_FILE_PATH =', SCORES_FILE_PATH)

BASE_DATA_DIR    = /content
INPUTS_PATH      = /content/inputs
OUT_DIR          = /content/outputs
SCORES_FILE_PATH = /content/outputs/clip_score.txt


## Import del progetto
Questo import include sia token-opt sia codice personale per generazione dei prompt (PrommptConfigurator), estrattore di immagini dalla cartella di input (ImageExtractor) e ImageSaver per creare l'immagine di output (immagine originale+immagine generata)

In [ ]:
from getpass import getpass
import os, pathlib, sys, zipfile, shutil, subprocess

!git clone https://github.com/niccobarto/token-opt-generator.git

REPO_DIR = Path('/content/token-opt-generator')
PERSONAL_PROJECT_DIR=Path('content/token-opt-generator/tester')
TOKEN_OPT_DIR= REPO_DIR/"token_opt"
sys.path.insert(0,str(TOKEN_OPT_DIR))
sys.path.insert(0,str(REPO_DIR))


Cloning into 'token-opt-generator'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 152 (delta 31), reused 134 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (152/152), 26.73 MiB | 19.90 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [ ]:
TTO_AVAILABLE = True
try:
    from tto.test_time_opt import(
        TestTimeOpt,
        TestTimeOptConfig,
        CLIPObjective,
    )
    print("Token-Opt importato correttamente")
except Exception as e:
    TTO_AVAILABLE = False
    print("Errore nell'importazione di Token-Opt:", e)

print(TTO_AVAILABLE)

TESTER_AVAILABLE=True
try:
  from tester.ImageExtractor import Extractor
  import tester.ImageSaver as ImageSaver
  from tester.PromptConfigurator import PromptConfigurator
  print("Tester importato correttamente")
except Exception as e:
  TESTER_AVAILABLE=False
  print("Errore nell'importazione di Tester",e)
print(TESTER_AVAILABLE)



Token-Opt importato correttamente
True
Tester importato correttamente
True


Per trasformare immagine in tensore e tensore in immagine

In [ ]:
def image_to_tensor(image:Image.Image, device=None):
  img = (1. / 255.) * torch.from_numpy(
      np.array(image).astype(np.float32)
  ).permute(2, 0, 1)
  img = tvf.resize(img, 256)
  img = tvf.center_crop(img, 256)
  img = img.unsqueeze(0)
  if device is not None:
      img = img.to(device)
  return img

def tensor_to_image(t: torch.Tensor) -> Image.Image:
    # Valori attesi in [0,1], su GPU o CPU
    t = t.detach().clamp(0, 1)

    # Se arriva [C,H,W], aggiungo la dim di batch -> [1,C,H,W]
    if t.ndim == 3:
        t = t.unsqueeze(0)
    elif t.ndim != 4:
        raise ValueError(f"tensor_to_image: tensore con {t.ndim} dimensioni non supportato")

    # [B,C,H,W] -> [H, B*W, C], uint8 su CPU
    img = Image.fromarray(
        rearrange((t * 255).to(dtype=torch.uint8, device="cpu"), "b c h w -> h (b w) c").numpy()
    )
    return img

# Metodo di generazione dell'immagine
`get_prompts()` utilizza PromptConfigurator per generare dei prompt in base alla categoria di modifica dell'immagine richiesta.

`get_images()` ricerca le immagini richieste nella cartella di inputs.

`run_token_opt()` esegue la generazione dell'immagine in base ad un'immagine di partenza ed il prompt associato. Il metodo trasforma l'immagine seed in tensore, la passa al token-opt e restituisce l'immagine riconvertita in RGB.

In [ ]:
def get_prompts(object_choose:str,category_choose:str,number_of_prompt:int,is_clean:bool):
  prompt_configurator=PromptConfigurator()
  prompts=prompt_configurator.make_prompts(object_choose,
                                          category_choose,
                                          number_of_prompt,
                                          is_clean,
                                          )
  return prompts

def get_images(obj:str)->tuple[list[Image.Image],list[Image.Image]]:
  extractor=Extractor(INPUTS_PATH)
  obj,clean_paths,real_paths=extractor.extract_paths(obj)
  obj,clean_images,real_images=extractor.extract_images(obj,clean_paths,real_paths)
  return clean_images,real_images

def run_token_opt(seed_img:Image.Image,prompt:str,cfg:TestTimeOptConfig)->Image.Image:
    """
    Esegue Token_Opt su una singola immagine+prompt. Se Token-Opt non è importato correttamente si restituisce l'immagine seed.
    """
    if not TTO_AVAILABLE:
        print("Token-Opt non è disponibile, restituisco l'immagine seed")
        return seed_img

    seed_tensor=image_to_tensor(seed_img,device="cuda")
    #Definizione dell'obiettivo CLIP
    objective.prompt=prompt
    print(f"Prompt {prompt} associato all'obiettivo")
    tensor_out,loss=tto(seed=seed_tensor)
    clip_score=(-loss.detach().mean()).item()
    return tensor_to_image(tensor_out),clip_score

# Loop principale

Creazione del TestTimeOptConfig, ovvero la configurazione per il TestTimeOpt.

Creazione dell'oggetto TestTimeOpt e dell'obiettivo CLIPObjective

In [ ]:
steps=301
cfg=TestTimeOptConfig(
        num_iter=steps,
        ema_decay=0.98,
        lr=1e-1,
        enable_amp=True,
        reg_weight=0.025,
        token_noise=0.3,
        reg_type="seed",
        vae_deterministic_sampling=False,

    )
objective=CLIPObjective(num_augmentations=8, cfg_scale=1.2)
tto=TestTimeOpt(cfg,objective).to(device)


Il metodo `process_images()` riceve una lista di immagini ed una lista di prompt. Per ogni immagine si esegue la generazione di token-opt con ogni prompt presente in `prompts`. Nel metodo una volta ricevuto l'immagine di output la valuto con ClipScore per misurare quando l'immagine assomiglia al prompt.
Restituisco per ogni prompt eseguito lo score associato a tale prompt.

In [ ]:
def process_images(images:list[Image.Image],image_type:str,obj:str,category:str):
  is_clean=True if image_type=="clean" else False
  clip_prompt_scores=[]
  number_dir=1
  for i in range(len(images)):
    number=1
    prompts=get_prompts(obj,category,prompts_per_image,is_clean)
    for prompt in prompts:
      output,clip_score=run_token_opt(images[i],prompt,cfg)
      clip_prompt_scores.append((clip_score,prompt))
      ImageSaver.create_side_by_side_with_caption(
              images[i],
              output,
              prompt=prompt,
              value=clip_score,
              out_path=OUT_DIR/f"{category}/{image_type}/{obj}/{obj}{number_dir}/{obj}{number}.jpg"
          )
      gc.collect()
      torch.cuda.empty_cache()
      number=number+1
    number_dir=number_dir+1

    gc.collect()
    torch.cuda.empty_cache()

  return clip_prompt_scores

Il metodo `process_categoty()` esegue su ogni oggetto deciso il testing di generazione sulla categoria `categoty` passata come parametro.

Oltre che scrivere i risultati su un file di testo, chiama il metodo `process_images()` su una lista di immagini raffiguranti lo stesso oggetto (e per oggetti con sfondo bianco e in contesti veri)

In [ ]:
def process_category(category:str):
  category_real_score=[]
  category_clean_score=[]

  with open(str(SCORES_FILE_PATH),"a",encoding="utf-8") as f:
    f.write(f"-------- {category} prompt testing --------\n")

  for obj in objects:
    real_scores=[]
    clean_scores=[]
    clean_images,real_images=get_images(obj)

    print(f"Processing real {obj} in {category} generation")

    #REAL IMAGES PROCESSING
    real_prompt_scores=process_images(real_images,"real",obj,category)
    real_rows=[]
    for score, prompt in real_prompt_scores:
        real_scores.append(score)
        real_rows.append(f"{category.upper()} REAL IMAGES generation. Prompt: {prompt}, Score: {score}\n")
    print(f"Done real {obj} on {category} generation")
    gc.collect()
    torch.cuda.empty_cache()
    print(f"Processing clean {obj} in {category} generation")

    #CLEAN IMAGES PROCESSING
    clean_prompt_scores=process_images(clean_images,"clean",obj,category)
    clean_rows=[]
    for score, prompt in clean_prompt_scores:
        clean_scores.append(score)
        clean_rows.append(f"{category.upper()} CLEAN IMAGES generation. Prompt: {prompt}, Score: {score}\n")

    print(f"Done clean {obj} on {category} generation")

    with open(str(SCORES_FILE_PATH),"a",encoding="utf-8") as f:
      f.writelines(real_rows)
      f.writelines(clean_rows)

    category_real_score.extend(real_scores)
    category_clean_score.extend(clean_scores)

  print(f"{category} generation DONE")
  gc.collect()
  torch.cuda.empty_cache()
  average_real=sum(category_real_score)/len(category_real_score)
  average_clean=sum(category_clean_score)/len(category_clean_score)
  end_category_real_row=f"The average ClipScore for {category} on real images is: {average_real}\n"
  end_category_clean_row=f"The average ClipScore for {category} on clean images is: {average_clean}\n"
  with open(str(SCORES_FILE_PATH),"a",encoding="utf-8") as f:
    f.write("\n")
    f.write(end_category_real_row)
    f.write(end_category_clean_row)
    f.write(f"-------- End {category} prompt testing --------\n")
  gc.collect()
  torch.cuda.empty_cache()

Scegli quali oggetti e categorie di generazione testare

In [ ]:
objects =["chair","cup","lamp","sofa","table","vase"]
categoryes = ["change_material", "change_color", "change_style", "add_component"]
prompts_per_image=2


for category in categoryes:
  process_category(category)

Processing real chair in change_material generation
Prompt a photo of a chair made of marble stone without changing background, high detail associato all'obiettivo
Prompt a photo of a chair made of black metal without changing background, high detail associato all'obiettivo
Prompt a photo of a chair made of polished steel without changing background, high detail associato all'obiettivo
Prompt a photo of a chair made of brushed aluminum without changing background, high detail associato all'obiettivo
Done real chair on change_material generation
Processing clean chair in change_material generation
Prompt a photo of a chair made of black metal on a white background, high detail associato all'obiettivo
Prompt a photo of a chair made of leather on a white background, high detail associato all'obiettivo
Prompt a photo of a chair made of carbon fiber on a white background, high detail associato all'obiettivo
Prompt a photo of a chair made of brushed aluminum on a white background, high detai

Comprimi la cartella di output per permettere il download

In [ ]:
!zip -r outputs /content/outputs

Cella bonus:
svuota la VRAM

In [ ]:
# PRIMA di ricreare modelli/oggetti
"""
import gc, torch, os

for name in ["tto", "objective"]:   # aggiungi qui eventuali altri handler GPU
    if name in globals():
        del globals()[name]

gc.collect()
torch.cuda.empty_cache()

